In [1]:
import numpy as np
from datetime import date, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import os
import math
import logging
from nsepython import *
import json
import csv, re

logging.basicConfig(level='CRITICAL')
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.CRITICAL)
mpl_logger.disabled = True

result = pd.DataFrame()
stage = 0
response = []

# https://www.nseindia.com/get-quotes/equity
# authority : www.nseindia.com
# path : /api/historical/cm/equity?symbol=SBIN&series=["EQ"]&from=26-03-2020&to=26-03-2021
# refer : https://www.nseindia.com/get-quotes/equity?symbol=SBIN

# set stock ticker symbol
symbol = 'SBIN'
series = 'EQ'
 
# set date range for historical prices
end_date = date.today() # 24-03-2021
start_date = end_date - timedelta(days=364) # 24-03-2020
 
# reformat date range
end = end_date.strftime('%d-%m-%Y')
start = start_date.strftime('%d-%m-%Y')


newheaders = {
            'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
            'DNT': '1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
            'Sec-Fetch-User': '?1',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
          }

extendedLink = "?symbol=" + symbol + "&series=[%22" + series + "%22]&from=" + start + "&to=" + end
stkurl = "https://www.nseindia.com/api/historical/cm/equity" + extendedLink +"&csv=true"


# stkHeaders = {
#     "Host": "www.nseindia.com",
#     "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0",
#     "Accept": "*/*",
#     "Accept-Language": "en-US,en;q=0.5",
#     "Accept-Encoding": "gzip, deflate, br",
#     "X-Requested-With": "XMLHttpRequest",
#     "Referer": "https://www.nseindia.com/get-quotes/equity?symbol="  + symbol,
#     "Path" : extendedLink,
#     "Access-Control-Allow-Origin": "*",
#     "Access-Control-Allow-Methods": "GET,POST,PUT,DELETE,OPTIONS",
#     "Access-Control-Allow-Headers": "Content-Type, Access-Control-Allow-Headers, Authorization, X-Requested-With",
#     'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
# }


In [2]:
try:
    output = requests.get("http://nseindia.com", headers=newheaders)
    s = requests.Session()
    output = s.get("http://nseindia.com",headers=newheaders)
    output = s.get(stkurl, headers=newheaders)
except ValueError:
    print(output)


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): nseindia.com:80
DEBUG:urllib3.connectionpool:http://nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.nseindia.com:80
DEBUG:urllib3.connectionpool:http://www.nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.nseindia.com:443
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET / HTTP/1.1" 200 38410
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): nseindia.com:80
DEBUG:urllib3.connectionpool:http://nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.nseindia.com:80
DEBUG:urllib3.connectionpool:http://www.nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.nseindia.com:443
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET / HTTP/1.1" 200 38408
DEBUG:urllib3.connectionpool

In [59]:
# print(type(output.content))
# chcp 65001
# set PYTHONIOENCODING=utf-8

data = output.content.decode('utf-8').splitlines()
# Open the file for writing
with open("tmp.csv", "w", encoding='utf-8') as csv_file:
    # Create the writer object with tab delimiter
#     writer = csv.writer(csv_file, delimiter = '\t')
    writer = csv.writer(csv_file, delimiter = ',', quoting=csv.QUOTE_NONE, quotechar='',escapechar='\\')
#     writer = csv.writer(csv_file, delimiter = ',', quotechar='"')
    for line in data:
        # Writerow() needs a list of data to be written, so split at all empty spaces in the line
#         print(line)
#         writer.writerow(line)
        writer.writerow(re.split('\s+',line))

    
# csv.register_dialect('csv_dialect',
#                     delimiter='|',
#                     skipinitialspace=True,
#                     quoting=csv.QUOTE_ALL)

df = pd.read_csv("tmp.csv", delimiter=',', quotechar='"', escapechar='\\',usecols=['Date,\\','close,\\'])
df['Date'] = df['Date,\\']
df['Date'] = df.Date.str.replace("\\","")
df['Close'] = df['close,\\']
df['Close'] = df.Close.str.replace("\\","").astype(float)
prices = df.drop(['Date,\\', 'close,\\'], axis=1)
print(prices)


            Date   Close
0    26-Mar-2021  357.20
1    25-Mar-2021  355.20
2    24-Mar-2021  359.85
3    23-Mar-2021  372.70
4    22-Mar-2021  367.00
..           ...     ...
244  07-Apr-2020  186.40
245  03-Apr-2020  175.50
246  01-Apr-2020  186.55
247  31-Mar-2020  196.85
248  30-Mar-2020  186.90

[249 rows x 2 columns]


In [61]:
prices['Date'] = pd.to_datetime( prices['Date'] )
# prices['Close'] = prices['Close'].astype(float)
prices.sort_values(by=['Date'], inplace=True, ascending=True)
# print(prices)

# calculate daily logarithmic return
prices['returns'] = (np.log(prices.Close / prices.Close.shift(-1)))

# calculate daily standard deviation of returns
# NumPy's std calculates the population standard deviation by default (it is behaving like Excel's STDEVP).
# To make NumPy's std function behave like Excel's STDEV, pass in the value ddof=1:
daily_std = round(np.std(prices.returns, ddof=1), 6)

# annualized daily standard deviation
samples = len(prices.index)-1
std = daily_std * round(math.sqrt( samples ), 4)
vix = np.round(std*100, 2)
print(vix)

42.01


In [ ]:
# # short term working solution using external scripts
# from NSEDownload import stocks
# # df = stocks.get_data(stockSymbol="SBIN", start_date = start, end_date = end)
# df = stocks.get_data(stockSymbol="SBIN", start_date = start, end_date = end)
# df = df['Close Price']
# print(df)

# pd.DataFrame(json_prices['data'])

# df = pd.DataFrame(r['data'])
# prices = df.rename(columns={'CH_TIMESTAMP':"Date", 'CH_CLOSING_PRICE':"Close"})
# prices['Date'] = pd.to_datetime( prices['Date'] )
# prices.sort_values(by=['Date'], inplace=True, ascending=True)
# print(prices)


# while(True):
# #     response = requests.get(indicsurl, headers=oldHeader)
# #     if (response.status_code == requests.codes.ok) :
#     page_content = BeautifulSoup(output.content, "html.parser")
#     df = pd.DataFrame(page_content)
#     print(pd)
#     print(df['Date'])
#     print(page_content)
#     try :
#         a = page_content.find(id="csvContentDiv").get_text();
#         a = a.replace(':', ", \n")
#         with open("stkdata.csv", "w") as f :
#             f.write(a)
# #         df = pd.read_csv("stkdata.csv",usecols=['Date','close'])
# #         result = pd.concat([result, df])
#     except AttributeError :
#         break;

#     else:
#         response.raise_for_status()
#     break;


# result = pd.read_csv(io.StringIO(output.content.decode('utf-8')))
# csvfile = output.content.decode('utf-8').replace('\\n', '\n').split(',')
# print(csvfile)
# f = open("demofile2.txt", "a")
# f.write(csvfile)
# f.close()
# print(output.content.decode('utf-8'))
# result = pd.read_csv(io.StringIO(output.content.decode('utf-8')))
# print(result)
csvfile = output.content.decode('utf-8').split(',')

for row in csvfile:
    print(row)
# data = [row for row in csvfile]
# print(data)
# result = pd.read_csv(data)
# print(result)
# prices = pd.DataFrame(result)
# result = result.drop(['series', 'OPEN', 'HIGH', 'LOW', 'PREV. CLOSE'])
# prices = prices.drop('series')
# 'OPEN', 'HIGH', 'LOW', 'PREV. CLOSE'
# print(prices['OPEN'])

In [ ]:
# Plot histograms
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
n, bins, patches = ax.hist(prices.returns.values, bins=50, alpha=0.65, color='blue')
 
ax.set_xlabel('log return of stock price')
ax.set_ylabel('frequency of log return')
ax.set_title('Historical Volatility for ' + symbol)
  
# get x and y coordinate limits
x_corr = ax.get_xlim()
y_corr = ax.get_ylim()
  
# make room for text
header = y_corr[1] / 5
y_corr = (y_corr[0], y_corr[1] + header)
ax.set_ylim(y_corr[0], y_corr[1])
 
# print historical volatility on plot
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 30
y = y_corr[1] - (y_corr[1] - y_corr[0]) / 15
ax.text(x, y , 'Annualized Volatility: ' + str(vix)+'%', fontsize=11, fontweight='bold')
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 15
y -= (y_corr[1] - y_corr[0]) / 20
 
# save histogram plot of historical price volatility
fig.tight_layout()
fig.savefig('historical volatility.png')
